In [ ]:
%reset

## 0. Set Paths and Imports

In [270]:
# SET PATHS

mainPath = 'C:/Users/cgiron/Documents/PROYECTO UB/'

#Path where raw data has been stored
rawDataPath = mainPath + "Raw_Data/UB_Features_V2/"

#Path where plots can be stored
pathToSavePics = mainPath + "Plots/"

pathCSV = 'C:/Users/cgiron/Documents/PROYECTO UB/ADHD_Project_UB/ADHD_Project/resultsClustering_sabado9.csv'

In [271]:
# IMPORTS

import numpy as np
import scipy.io as sio
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import mne
from mne.datasets import sample
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from mne.connectivity import spectral_connectivity
from mne.viz import circular_layout, plot_connectivity_circle


## 1. Load Data

Load data Fuction

In [288]:
#LOAD DATA: path where files have been stored need to be set before loading this script
def load( path ):
    #columns for the band powers
    filenames = os.listdir(path)

    #electrodes names
    electrodes = ['Fp1', 'F3', 'C3', 'Fz', 'Cz', 'Fp2', 'F4', 'C4']

    #bandPower names
    bandPower = ['Theta2+Alpha1', 'Theta', 'Alpha', 'Beta_Global', 'Beta_Alta', 'Beta_Baja', 'Gamma']

    #let us create the real dataframe
    #columns for the band powers
    columns_df=[]
    for i in electrodes:
        for j in bandPower:
            combination = i + '_(' + j + ')'
            columns_df.append(combination)

    #columns for the band power ratios
    columns_df2 = ['BPR_Fp1', 'BPR_F3', 'BPR_C3', 'BPR_Fz', 'BPR_Cz', 'BPR_Fp2', 'BPR_F4', 'BPR_C4']

    #patterns to determine experiments
    patternA = r'[\w]+A\.mat'
    patternB = r'[\w]+B\.mat'
    patternC = r'[\w]+C\.mat'

    #define the list of dataframes
    list_df=[]
    for i in filenames:
        patient = sio.loadmat(path + i)
        bandPowerDF = pd.DataFrame(patient['BandPower'].reshape(1,56), columns = columns_df)
        bandPowerRatioDF = pd.DataFrame(patient['BandPowerRatio'].reshape(1,8), columns = columns_df2)
        patientDF =  pd.concat([bandPowerDF, bandPowerRatioDF], axis=1)
        #patientDF = pd.DataFrame(patient['BandPower'].reshape(1,56), columns = columns_df)
        patientDF['patientName'] = i.split('.mat')[0]
        
        if re.match(patternA,i) != None:
            patientDF['experiment'] = 'A'
        elif re.match(patternB,i) != None:
            patientDF['experiment'] = 'B'
        elif re.match(patternC,i) != None:
            patientDF['experiment'] = 'C'
                
        list_df.append(patientDF)

    #via the concat all the dataframes are concatenated
    patientsDF = pd.concat(list_df)
    #nans are dropped
    patientsDF = patientsDF.dropna(how='any')

    #dataframe is sorted by experiment type: order A, B and C
    patientsDF.sort_values(by='experiment', ascending= True, inplace=True)

    #indexs are reset
    #patientsDF = patientsDF.reset_index(drop = True)

    #indexs are set according to the patient file
    patientsDF.set_index(['patientName'], inplace=True)
    #Return resulting data frame with all patients
    return ( patientsDF )

In [289]:
patientsDF = load ( rawDataPath )
patientsDF.head()

,Fp1_(Theta2+Alpha1),Fp1_(Theta),Fp1_(Alpha),Fp1_(Beta_Global),Fp1_(Beta_Alta),Fp1_(Beta_Baja),Fp1_(Gamma),F3_(Theta2+Alpha1),F3_(Theta),F3_(Alpha),...,C4_(Gamma),BPR_Fp1,BPR_F3,BPR_C3,BPR_Fz,BPR_Cz,BPR_Fp2,BPR_F4,BPR_C4,experiment
patientName,,,,,,,,,,,,,,,,,,,,,
S29_A,3.603942,4.060272,2.051598,0.412040,0.188472,0.493966,0.160718,3.512098,3.725556,1.929716,...,0.162646,21.142656,16.521217,18.411389,20.192714,20.738564,12.910780,15.919647,19.688982,A
S74_A,2.853595,3.694020,1.676661,0.549950,0.338246,0.567536,0.306302,3.303229,3.881861,2.130401,...,0.124892,8.883901,21.721696,33.660980,2.572054,2.518262,13.364211,15.710850,29.045320,A
S18_A,2.376608,3.137197,1.373746,0.791417,0.446905,0.843949,0.402894,2.473590,3.208466,1.510230,...,0.305643,5.871080,7.145010,11.185255,8.860686,15.966705,4.262830,7.567438,9.950759,A
S75_A,3.100145,3.696751,1.348613,0.635941,0.445743,0.553323,0.371787,3.161179,3.665984,1.415384,...,0.249343,7.661245,7.917380,12.437474,12.007422,15.961836,7.391989,9.201975,14.733004,A
S119_A,2.977011,4.400604,1.274504,0.509350,0.316565,0.520461,0.289575,2.921852,3.875402,1.397294,...,0.342864,11.157300,9.495439,10.272425,17.101101,13.740270,4.384690,10.244064,8.974231,A


Normalization data function

In [276]:
#Normalize data per experiment
def normalizeData ( dfValues ):
 #   dfValues = patientsDF.drop( ['experiment'],1 ).copy()
    patientsNames = dfValues.index
    columnsNames = dfValues.columns

    dfNorm = StandardScaler().fit_transform( dfValues )
    dfResult = pd.DataFrame( dfNorm, index = patientsNames, columns = columnsNames )

    return ( dfResult )


In [290]:
dataFrame = patientsDF.drop('experiment', 1).copy()
normsPatientsDF =  normalizeData ( dataFrame ).copy()
normsPatientsDF.head()

,Fp1_(Theta2+Alpha1),Fp1_(Theta),Fp1_(Alpha),Fp1_(Beta_Global),Fp1_(Beta_Alta),Fp1_(Beta_Baja),Fp1_(Gamma),F3_(Theta2+Alpha1),F3_(Theta),F3_(Alpha),...,C4_(Beta_Baja),C4_(Gamma),BPR_Fp1,BPR_F3,BPR_C3,BPR_Fz,BPR_Cz,BPR_Fp2,BPR_F4,BPR_C4
patientName,,,,,,,,,,,,,,,,,,,,,
S29_A,1.434702,1.051774,0.625946,-1.158842,-1.162347,-0.970072,-1.189749,1.119020,0.400895,0.247692,...,-0.804493,-0.678916,1.754175,0.633761,0.226787,0.719597,0.142898,0.662027,0.754757,0.297203
S74_A,0.474059,0.594589,0.007963,-0.491935,-0.399176,-0.544209,-0.374841,0.853876,0.595986,0.564442,...,-0.044963,-0.940963,-0.011252,1.248653,1.455439,-1.138062,-1.157742,0.728763,0.727000,0.950446
S18_A,-0.136609,-0.100480,-0.491311,0.675756,0.154498,1.055843,0.165834,-0.199286,-0.244506,-0.414398,...,0.607412,0.313605,-0.445139,-0.474857,-0.355419,-0.475083,-0.197737,-0.610789,-0.355580,-0.382703
S75_A,0.789708,0.597998,-0.532735,-0.076098,0.148576,-0.626480,-0.008285,0.673555,0.326541,-0.564096,...,-0.854436,-0.077165,-0.187331,-0.383534,-0.254528,-0.143338,-0.198084,-0.150235,-0.138286,-0.048815
S119_A,0.632064,1.476603,-0.654884,-0.688268,-0.509649,-0.816704,-0.468467,0.369748,0.587924,-0.592649,...,0.058331,0.571948,0.316148,-0.196948,-0.428965,0.393663,-0.356669,-0.592853,0.000249,-0.450882


Add cluster label

In [293]:
def addLabel ( pathCSV, dataFrame ):
    dfClustering = pd.read_csv( pathCSV )
    dfClustering.index = dfClustering['patientName']
    result = dataFrame.reset_index().merge( dfClustering, how = 'inner').set_index('patientName')
    return ( result )

In [294]:
finalPatientsDF = addLabel ( pathCSV, dataFrame ).copy()
finalPatientsDF.head()

,Fp1_(Theta2+Alpha1),Fp1_(Theta),Fp1_(Alpha),Fp1_(Beta_Global),Fp1_(Beta_Alta),Fp1_(Beta_Baja),Fp1_(Gamma),F3_(Theta2+Alpha1),F3_(Theta),F3_(Alpha),...,BPR_C4,Unnamed: 0,experiment,cluster_GMM,cluster_Hierarchichal,cluster_Kmeans,cluster_Spectral,PCA_X,PCA_Y,Best_Cluster
patientName,,,,,,,,,,,,,,,,,,,,,
S29_A,3.603942,4.060272,2.051598,0.412040,0.188472,0.493966,0.160718,3.512098,3.725556,1.929716,...,19.688982,144,A,0,0,0,0,9.316071,2.086899,0
S74_A,2.853595,3.694020,1.676661,0.549950,0.338246,0.567536,0.306302,3.303229,3.881861,2.130401,...,29.045320,264,A,0,0,0,0,3.745638,5.466005,0
S18_A,2.376608,3.137197,1.373746,0.791417,0.446905,0.843949,0.402894,2.473590,3.208466,1.510230,...,9.950759,114,A,1,1,1,1,-0.645384,2.287421,1
S75_A,3.100145,3.696751,1.348613,0.635941,0.445743,0.553323,0.371787,3.161179,3.665984,1.415384,...,14.733004,267,A,0,0,0,0,4.036546,-1.076673,0
S119_A,2.977011,4.400604,1.274504,0.509350,0.316565,0.520461,0.289575,2.921852,3.875402,1.397294,...,8.974231,48,A,0,0,0,0,2.660355,-1.977821,0


## 2. PairPlots

In [207]:
# %load pairPlots.py
## Pair plots per electrode and experiment (including bandPower at the end)
#electrodes: List of electrodes to be ploted
#bandPower: List of bands to be ploted
#pathToSavePlot: path where to save pairplots

def pairPlotsPerExperiment ( dataFrame, electrodes, bandPower, pathToSavePlot, title ):
    patientsDF =  normalizeData ( dataFrame )
    bandColumns = [ ]
    bandPowerColumns = [ ]
    for i in electrodes:
        for j in bandPower:
            combinationBand = i + '_(' + j + ')'
            bandColumns.append( combinationBand )
            combinationPower = 'BPR_' + i 
            bandPowerColumns.append( combinationPower )
    numberOfBandPower = len ( bandPower )
    
    for i in range( 0, len( bandColumns ) - 1, numberOfBandPower ):
        electrodePossition = int( (i + 1 ) / numberOfBandPower )
        electrodeBand =  bandColumns[ i : i + numberOfBandPower ]
        electrodeBand.append( bandPowerColumns[ electrodePossition ] )
        df = patientsDF[ electrodeBand ]
        plot = sns.pairplot(df)
        electrode = electrodes[ electrodePossition ]
        picPath = pathToSavePlot + electrode + title + ".png"
        plot.savefig( picPath )
        print ("Pair-plot for the electrode " + electrode +  " has been saved")
        plt.close('all')

######
## HOW TO USE IT: EXAMPLE
##patientsDF = loadData( path )
##electrodes = ['Fp1', 'F3', 'C3', 'Fz', 'Cz', 'Fp2', 'F4', 'C4']
##bandPower = ['Theta', 'Theta2+Alpha1', 'Alpha', 'Beta_Global']
##pathToSavePlot = mainPath + "Plots/"
##pairPlotsPerExperiment ( patientsDF, electrodes, bandPower, pathToSavePlot )
##
######

In [ ]:
dataFrame = patientsDF[patientsDF.experiment == 'A'].copy()
electrodes = ['Fp1', 'F3', 'C3', 'Fz', 'Cz', 'Fp2', 'F4', 'C4']
bandPower = ['Theta', 'Theta2+Alpha1', 'Alpha', 'Beta_Global', 'Gamma']
pathToSavePlot = mainPath + "Plots/"
title = "expA"
pairPlotsPerExperiment ( dataFrame, electrodes, bandPower, pathToSavePlot, title )

## 3. Correlation graph plot

In [303]:
def corrGraphPlot( dataFrame, electrodes, bandPower, numberOfEdges, plotTitle ):
    
    #Create columns for the band powers
    label_names=[]
    for i in electrodes:
        for j in bandPower:
            combination = i + '_(' + j + ')'
            label_names.append(combination)
        
    node_angles = circular_layout(label_names, label_names, start_pos=90, group_boundaries=[0, len(label_names) / 2 ])
    # Dataframe to plot correlations
    valuesDF = dataFrame[label_names]

    # Create Correlation matrix
    corrDF = valuesDF.corr()
    corrValues = corrDF.values
    
    #Set size nodes depending on their amounmt of correlation with other nodes
    #dfAbs = corrDF.abs()
    #absoluteValues = np.absolute(corrDF.values)
    #sumAbsValues =  dfAbs.sum()/a.sum()
    #sizeNodes = sumAbsValues.values*50

    # Plot correlationGraph
    plot = plot_connectivity_circle( corrValues, 
                             label_names, 
                             n_lines = numberOfEdges,
                             node_angles=node_angles,
                             colormap  = 'RdYlGn',
                             facecolor ='White',
                             textcolor = 'Black',
                             node_width = 3,
                             title = plotTitle )

######
## HOW TO USE IT: EXAMPLE
##
#patientsDF = load ( rawDataPath )
#dataFrame = addLabel ( pathCSV, patientsDF ).copy()
#dataFrame = dataFrame[dataFrame['experiment'] == 'A']
#dataFrame = dataFrame[dataFrame.Best_Cluster == 1]
#electrodes = ['Fp1', 'F3', 'C3', 'Fz', 'Cz', 'Fp2', 'F4', 'C4']
#bandPower = ['Theta2+Alpha1', 'Theta', 'Alpha', 'Beta_Global', 'Gamma']
#experiment = 'A'
#numberOfEdges = 100
#plotTitle = 'All-to-All Correlation - Experiment A'
#corrGraphPlot( dataFrame, electrodes, bandPower, numberOfEdges, plotTitle )
##
######

In [345]:
patientsDF = load ( rawDataPath )
dataFrame = addLabel ( pathCSV, patientsDF ).copy()
dataFrame = dataFrame[ dataFrame['experiment'] == 'C' ]
dataFrame = dataFrame[ dataFrame.Best_Cluster == 1 ]

electrodes = [ 'F3', 'C3', 'Fz', 'Cz', 'F4', 'C4']
bandPower = ['Theta2+Alpha1', 'Theta', 'Alpha', 'Beta_Global', 'Gamma']
numberOfEdges = 80
plotTitle = 'All-to-All Correlation - Experiment C - Label Patient 1'

corrGraphPlot( dataFrame, electrodes, bandPower, numberOfEdges, plotTitle )

Graph representing how different are correlation between electrode_band's for each label patient

In [346]:
def diffBetweenLabelsCorrGraphPlot( dataFrame, electrodes, bandPower, numberOfEdges, plotTitle ):
    
    #Create columns for the band powers
    label_names=[]
    for i in electrodes:
        for j in bandPower:
            combination = i + '_(' + j + ')'
            label_names.append(combination)

    node_angles = circular_layout(label_names, label_names, start_pos=90, group_boundaries=[0, len(label_names) / 2 ])
    # Dataframe to plot correlations

    # Dataframe label 1
    valuesDF_l1 = dataFrame[ dataFrame.Best_Cluster == 1 ]
    valuesDF_l0 = dataFrame[ dataFrame.Best_Cluster == 0 ]
    # Create Correlation matrix
    corrDF_l1 = valuesDF_l1[label_names].corr()
    corrDF_l0 = valuesDF_l0[label_names].corr()
    diffCorrDF = corrDF_l1 - corrDF_l0
    diffCorrValue = np.absolute(diffCorrDF.values)
    plot = plot_connectivity_circle( diffCorrValue, 
                                 label_names, 
                                 n_lines = numberOfEdges,
                                 node_angles=node_angles,
                                 colormap  = 'OrRd',
                                 facecolor ='White',
                                 textcolor = 'Black',
                                 node_width = 3,
                                 title = plotTitle )


In [351]:
patientsDF = load ( rawDataPath )
dataFrame = addLabel ( pathCSV, patientsDF ).copy()
dataFrame = dataFrame[ dataFrame['experiment'] == 'A' ]

electrodes = [ 'F3', 'C3', 'Fz', 'Cz', 'F4', 'C4']
bandPower = ['Theta2+Alpha1', 'Theta', 'Alpha', 'Beta_Global', 'Gamma']
plotTitle =  'Absolute difference between correlations - Experiment A'
numberOfEdges = 435

# Plot correlationGraph
diffBetweenLabelsCorrGraphPlot( dataFrame, electrodes, bandPower, numberOfEdges, plotTitle )